# [TLDR] `sklearn.neighbors` logic related to `kneighbors` gathered here

# [LONGER VERSION]
- want to get `similar points` a set of `source points`
- `source points`
  - refers to products that got matched with text token grepping
  - they are the starting points of the search extension
- `similar points`
  - refers to non text matched products that in some way is close to the `source points`
- the final approach taken is to use `kneighbors` on normalized numeric data from the dataset to find `similar points`
  - I toyed with kmeans and agglomerative clustering before deciding on `kneighbors`

# IMPORTS

In [ ]:
%run ipynb_setup.ipynb

In [ ]:
%run class_Dataset.ipynb

In [ ]:
from inspect import currentframe 
from typing import List
from sklearn.neighbors import NearestNeighbors

# CLASS DEF

In [ ]:
class NeighbourSearch():

    def __init__(
        self,
        dataset   : Dataset,
        verbose   : int = 0,
        ) -> None :

        self.dataset   = dataset
        self.model     = NearestNeighbors().fit(self.dataset.df_num)    
    
    # given list of locs, return a larger list of size `n` x len(locs) of locs that are closest to original locs
    def get_n_nearest_from_locs(
        self,
        n_nearest : int,
        locs      : List[int],
        verbose   : int = 0,
        ) -> pd.Series :
        if verbose>0: print(f'[{currentframe().f_code.co_name}] n_nearest = {n_nearest}')
        if verbose>0: print(f'[{currentframe().f_code.co_name}] locs = {locs}')
        if verbose>0: print(f'[{currentframe().f_code.co_name}] df_num = {self.dataset.df_num}, type = {type(self.dataset.df_num)}')
        
        # get nearest_n neighbors from each element of locs 
        distances,indices=self.model.kneighbors(
            self.dataset.df_num.loc[locs],
            n_neighbors = n_nearest,
        )
        
        # merge results into single vector
        flat_distances = np.ravel(distances)
        flat_ilocs     = np.ravel(indices)
        flat_locs      = self.dataset.df_num.index[flat_ilocs]
        #nearest_results_ilocs = pd.Series(flat_distances,index=flat_ilocs).sort_values(ascending=True) # ordered from nearest to furthest, want globally nearest to origin family
        nearest_results_locs = pd.Series(flat_distances,index=flat_locs).sort_values(ascending=True) # ordered from nearest to furthest, want globally nearest to origin family
        
        # return
        #return nearest_results_ilocs
        return nearest_results_locs
    
    # incrementally grab more nearest neighbours from some `source_results` list until we get more than we need
    def nearby_results(
        self,
        locs        : List[int],
        min_results : int,
        verbose     : int = 0,
        ) -> pd.Series :
            
        # protect input
        if min_results > len(self.dataset.df): min_results = len(self.dataset.df)
            
        # initialize while loop
        n_nearest = 1
        res_unique = locs
        
        # look outwards and find nearest results (including self) until we get more that what we need
        while len(res_unique) < min_results:
            # get next nearest result from each source_result
            res = self.get_n_nearest_from_locs(
                n_nearest = n_nearest,
                locs      = locs,
            )
            
            # avoid situation where same neighbor is found jas
            res_unique = res.groupby(res.index).min()
        
            if verbose>0: print(f'[{currentframe().f_code.co_name}] n_nearest = {n_nearest}, len(res) = {len(res)}, len(res_unique) = {len(res_unique)}, min_results = {min_results}')

            # prep for next loop
            n_nearest = n_nearest + 1
            
        # return required number of results sorted by distance away from original locs
        return res_unique.sort_values()

In [ ]:
'''
d=Dataset()
neighbour_search=NeighbourSearch(dataset=d)
#neighbour_search.get_n_nearest_from_locs(n_nearest=5,locs=[1483, 1307, 1362, 1485, 172, 375])
#neighbour_search.get_n_nearest_from_locs(n_nearest=1,locs=[172, 375, 1307, 1362, 1483, 1485])
#neighbour_search.get_n_nearest_from_locs(n_nearest=5,locs=[0,1,2])
#neighbour_search.get_n_nearest_from_locs(n_nearest=5,locs=[0,1,2,3,4,5,6])
#neighbour_search.get_n_nearest_from_locs(n_nearest=100,locs=[0,1,2,3,4,5,6])
#neighbour_search.nearby_results(locs=[0,2,4,6],min_results=10000) # ensure while loop finishes
neighbour_search.nearby_results(locs=[0,2,4,6],min_results=10) # ensure while loop finishes
#'''
None

In [ ]:
#neighbour_search.get_n_nearest_from_locs(n_nearest=2,locs=d.df.index[:4])
#neighbour_search.nearby_results(locs=d.df.index[:4],min_results=20) # ensure while loop finishes